## This notebook requires GPU

This lab must be run in Google Colab in order to use GPU acceleration for model training. Click the button below to open this notebook in Colab, then set your runtime to GPU:

**Runtime > Change Runtime Type > T4 GPU**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scott2b/coursera-msds-public/blob/main/notebooks/MSDSTextClassification_Lab2_KTrain.ipynb)

# Text Classification, Lab 2: Building a model with K-Train

## About this lab

The first part of this lab is identical to Lab 1, and you should be able to step through it easily.

We continue now with the goal of building an inference model for predicting whether or not a document is about "healthy living." We will use the K-Train library to do this.

## About the final project


Recall that you are working toward a final project. After completing this lab, you will want to go the extra mile and explore ways to tweak and improve your model. See the final project description for further details on what is expected.

## Data Setup

The data file for this lab is available in the Coursera lab environment in the `data/` directory. To use it in Colab:

1. In the Coursera lab, click the **Files** tab and navigate to `lab2 / data`
2. Select the checkbox next to `news_category_trainingdata.json`, then click the **Download** button in the toolbar
3. In your Google Drive, create a folder called `coursera-msds` (if you haven't already)
4. Upload the downloaded file to `coursera-msds/`

The code below will mount your Google Drive and load the data automatically.

## Imports

We're going to be using Google's Tensorflow package:
https://www.tensorflow.org/tutorials

We're using an API wrapper for Tensorflow called ktrain. It's absolutely fabulous because it really abstracts the whole deep learning process into a workflow so easy, even a computational social scientist can do it:
https://github.com/amaiya/ktrain

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'

try:
    from transformers import TFAutoModel  # removed in transformers v5
    import ktrain
    from ktrain import text
except ImportError:
    # ktrain needs TF model classes from transformers v4.x
    !pip uninstall -y transformers
    !pip install "transformers>=4.43,<5" "numpy<2" ktrain tf-keras
    os.kill(os.getpid(), 9)  # restart runtime; cell re-runs cleanly

import ktrain
import numpy as np
np.Inf = np.inf  # compatibility shim: np.Inf removed in NumPy 2.0
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load the data

In [ ]:
DATA_FILENAME = 'news_category_trainingdata.json'
DRIVE_PATH = f'/content/drive/MyDrive/coursera-msds/{DATA_FILENAME}'
LOCAL_PATH = f'data/{DATA_FILENAME}'

if os.path.exists(DRIVE_PATH):
    data_path = DRIVE_PATH
    print(f'Loading data from Google Drive: {DRIVE_PATH}')
elif os.path.exists(LOCAL_PATH):
    data_path = LOCAL_PATH
    print(f'Loading data from local path: {LOCAL_PATH}')
else:
    raise FileNotFoundError(
        f'Data file not found. Please either:\n'
        f'  1. Upload {DATA_FILENAME} to Google Drive at: MyDrive/coursera-msds/\n'
        f'  2. Place {DATA_FILENAME} in a local data/ directory'
    )

reviews = pd.read_json(data_path)

## Inspect the data

In [ ]:
reviews.head()

## Prepare the data

Most machine learning tools in Python accept one field/column/string. So we have to merge our two text column. Let's separate it with a space.

In [ ]:
reviews['combined_text'] = reviews['headline'] + ' ' + reviews['short_description']

The first thing we need to do is prepare the data. Specifically, we have a categorical column that we want to turn into a "is this article healthy living?" column. That is, when an article is about healthy living, it should have a 1, when it's anything else, it should be a 0.

In [ ]:
reviews[reviews['category'].str.contains("HEALTHY LIVING")]

In [ ]:
reviews['healthy'] = np.where((reviews['category'] == 'HEALTHY LIVING'), 1, 0)

In [ ]:
reviews['healthy'].describe()

## Balance the data

Our data is very unbalanced — there are far more non-health articles than health articles. We balance the dataset by sampling equal instances of both classes.

Set `SAMPLE_PER_CLASS` to `None` to use the full balanced dataset (slower but potentially higher accuracy). The default of 1000 keeps training quick on a T4 GPU. After completing the lab, consider increasing the sample size to see if you can improve model performance.

In [ ]:
# Balance the dataset by sampling equal instances of both classes.
# Set SAMPLE_PER_CLASS to None to use the full dataset (slower but
# potentially higher accuracy).
SAMPLE_PER_CLASS = 1000  # set to None for full dataset

healthy = reviews[reviews['healthy'] == 1]
not_healthy = reviews[reviews['healthy'] == 0]

if SAMPLE_PER_CLASS is not None:
    healthy = healthy.sample(n=min(SAMPLE_PER_CLASS, len(healthy)), random_state=42)
    not_healthy = not_healthy.sample(n=min(SAMPLE_PER_CLASS, len(not_healthy)), random_state=42)

sample_amount = min(len(healthy), len(not_healthy))
healthy = healthy.head(sample_amount)
not_healthy = not_healthy.head(sample_amount)
review_sample = pd.concat([healthy, not_healthy])
print(f'Training with {len(review_sample)} samples ({sample_amount} per class)')

In [ ]:
review_sample.describe()

# On to Lab 2: Test, Tune and Save Models

Here, you will tune and train a predictor model for classifying healthy-living articles. After completing this lab, complete the Lab Quiz by entering your precision and recall values from the validation report for both the negative and positive classes.

In [ ]:
target_names = ['NOT HEALTHY LIVING','HEALTHY LIVING']

---

### Experimenting with different transformers

For purposes of this lab, we are using the **distilbert-base-uncased** transformer model. Other models you might try for your final project include:

 * roberta-base
 * bert-base-uncased
 * distilroberta-base

See all the models here: https://huggingface.co/transformers/pretrained_models.html

Some work, some dont, try at your own risk.

---

In [ ]:
train, val, preprocess = ktrain.text.texts_from_df(
    review_sample,
    "combined_text",
    label_columns=["healthy"],
    val_df=None,
    max_features=20000,
    maxlen=512,
    val_pct=0.1,
    ngram_range=1,
    preprocess_mode="distilbert",
    verbose=1
)

In [ ]:
model = preprocess.get_classifier()
learner = ktrain.get_learner(model, train_data=train, val_data=val, batch_size=16)

In [ ]:
learner.lr_find(max_epochs=6)

In [ ]:
learner.lr_plot()

Now, use the tuned learner to train the best model.

Here, we define a limit of 10 epochs, but in reality, this should stop much sooner due to early stopping.

In [ ]:
history=learner.autofit(
    1e-4,
    checkpoint_folder='checkpoint',
    epochs=10,
    early_stopping=True
)

Get the predictor

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=preprocess)

Optionally, uncomment this code to save the predictor and reload it later. Note, the saved models can be quite large and may quickly use up space on your Google Drive.

In [ ]:
#predictor.save("drive/MyDrive/MSDSTextClassification_Lab2.healthy_living")

In [ ]:
validation = learner.validate(val_data=val, print_report=True)

---

## 🧐 Lab Quiz Questions 1-4

Enter the following values from the validation report into the Lab Quiz:

 1. precision for non healthy-living articles
 2. recall for non healthy-living articles
 3. precision for healthy-living articles
 4. recall for healthy-living articles

 ---


Keep in mind that we've reduced the training set for the sake of expediency. For your final analysis and project, you should complete a run of the full data set. Pay attention to the impact of the input data on the performance of the final model (i.e. the validation scores)

# Inspecting the drivers of prediction

No matter what the supervised machine learning model, you always want to peak under the hood to see what features are driving prediction. That is, what words sway the outcome of the prediction. It's harder to inspect a neural network. Because all of the layers of a neural network aren't really interpretable to the human eye.

Currently, the best practice I've found is a little tool Explainable AI:
https://alvinntnu.github.io/python-notes/nlp/ktrain-tutorial-explaining-predictions.html

In [ ]:
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1

Let's go ahead and make a little set of test documents to check out

In [ ]:
test_docs = [
'Stress May Be Your Heart’s Worst Enemy Psychological stress activates the fear center in the brain, setting into motion a cascade of reactions that can lead to heart attacks and strokes.',
'Exercising to Slim Down? Try Getting Bigger. It’s high time for women to reclaim the real strength behind exercise.',
'What Are Your Food Resolutions for the New Year? Join us for the Eat Well Challenge starting in January.',
'Why We All Need to Have More Fun. Prioritizing fun may feel impossible right now. But this four-step plan will help you rediscover how to feel more alive.',
'Cuomo Will Not Be Prosecuted in Groping Case, Albany D.A. Says. The district attorney described the woman who said former Gov. Andrew Cuomo had groped her as “credible,” but added that proving her allegation would be difficult.',
'A Film Captures Jewish Life in a Polish Town Before the Nazis Arrived. A documentary based on a home movie shot by an American in 1938 provides a look at the vibrancy of a Jewish community in Europe just before the Holocaust.'
             ]

In [ ]:
for i, text in enumerate(test_docs):
  probs = predictor.predict(text, return_proba=True)
  print("---------------------------")
  print('The probability this is healthy is %s' % probs[1])
  print(text)

*These* are pretty obvious examples, but it works exactly as expected!

In [ ]:
predictor.explain('Diversity is the key to a healthy society. Here is what we need to do to make america a more equitable place to live for all.')

But you can see, this algorithm is far from perfect. Here you can see that it's probably got too high of an emphasis on the word "healthy."

So what would I do next? Well, given that it's over reacting to worrds like health and equitable, I'd try introducing more negative examples into the data, times where healthy is used outside of health and wellness news. We can do this by changing our sample from 50/50 to something like 20/80, but of course, the more documents we process, the longer this model is going to take to make!